[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_Code/01_Highway_Env/04_Edge_Case_Scenario_2/03_BIASED_HF_DIRECT_RAD/02_Model_testing_Biased_HF_D_RECKLESS_ADAPTIVE_Edge_case_2.ipynb)

In [ ]:
# Install the required libraries in your Google Colab environment
!pip install gymnasium stable-baselines3 highway-env -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
# Import the necessary libraries
import gymnasium as gym
import highway_env
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from google.colab import drive
from google.colab import data_table

# Mount Google Drive
drive.mount('/content/drive')

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Mounted at /content/drive


CALCULATE MISALIGNMENT FUNCTION

In [ ]:
def calculate_misalignment(df, reward_threshold=0, lane_change_penalty=1, collision_penalty=5, reward_penalty=2):
    """
    Calculate misalignment score for each episode in the dataset.

    Parameters:
        df (pd.DataFrame): The data frame containing highway environment trajectories.
        reward_threshold (float): Threshold below which reward is considered suboptimal.
        lane_change_penalty (float): Penalty factor for unnecessary lane changes.
        collision_penalty (float): Penalty factor for collisions.
        reward_penalty (float): Penalty factor for suboptimal rewards.

    Returns:
        pd.DataFrame: Data frame with misalignment scores per episode.
    """
    misalignment_scores = []  # Changed to 'misalignment_scores' to store all episode scores

    for episode in df['episode'].unique():
        episode_df = df[df['episode'] == episode].sort_values(by='time_step')

        # Calculate lane changes
        lane_changes = episode_df['lane_index'].diff().abs().sum()

        # Count collisions
        collisions = episode_df['collision_flag'].sum()

        # Count suboptimal rewards
        suboptimal_rewards = (episode_df['reward'] < reward_threshold).sum()

        # Compute misalignment score
        misalignment_score = (
            lane_change_penalty * lane_changes +
            collision_penalty * collisions +
            reward_penalty * suboptimal_rewards
        )

        misalignment_scores.append({'episode': episode, 'Misalignment Score': misalignment_score})

    df = pd.DataFrame(misalignment_scores) # Create the DataFrame outside the loop

    # Calculate the overall misalignment as the mean of per-episode misalignment
    Average_misalignment = df['Misalignment Score'].mean() # Calculate mean of 'Misalignment Score' column

    return df, Average_misalignment

AVERAGE TERMINATE TIME (ATT) FUNCTION

In [ ]:
def calculate_collision_rate(test_trajectory_df):
    # Initialize an empty list to store the results
    results = []

    # Group by 'episode' to process each episode separately
    episodes = test_trajectory_df.groupby('episode')

    # Iterate through each episode
    for episode, episode_data in episodes:
        # Count the number of collisions in the current episode
        num_collisions = episode_data['collision_flag'].sum()

        # Get the total number of timesteps in the current episode
        total_steps = len(episode_data)

        # Append the results to the list
        results.append({
            'Episode': episode,
            'Total Steps': total_steps
        })

    # Convert the list of results to a DataFrame
    results_df = pd.DataFrame(results)

    # Calculate the overall Collision rate as the mean of per-episode Collision rate
    Average_terminate_time = results_df['Total Steps'].mean()

    # Return the resulting DataFrame
    return results_df, Average_terminate_time

STABILITY OF THE LEARNING CURVES OF THE MODEL FUNCTION

In [ ]:
def analyze_learning_curve(test_trajectory_df):
    # Step 1: Calculate Cumulative Reward Per Episode
    cumulative_rewards = test_trajectory_df.groupby('episode')['reward'].sum().reset_index()
    cumulative_rewards.columns = ['episode', 'cumulative_reward']

    # Step 2: Calculate Smoothed Rewards using Moving Average (window size of 10 episodes)
    window_size = 10
    cumulative_rewards['smoothed_reward'] = cumulative_rewards['cumulative_reward'].rolling(window=window_size, min_periods=1).mean()

    # Step 3: Calculate Variance and Standard Deviation of Cumulative Rewards
    variance = cumulative_rewards['cumulative_reward'].var()
    std_dev = cumulative_rewards['cumulative_reward'].std()
    final_reward = cumulative_rewards['cumulative_reward'].mean()

    # Step 4: Return the dataframe containing all results
    return cumulative_rewards, variance, std_dev, final_reward

COMBINING ALL THE DATA FRAMES

In [ ]:
def create_cummulative_reward(df1, df2, df3, df4, df5):
    """
    Create a data frame 'ideal_cummulative_reward' that contains:
      - 'episode' column (common across all data frames)
      - 'cumulative_reward_1' to 'cumulative_reward_5' columns from each data frame respectively,
        where each input data frame has columns 'episode' and 'cummulative_reward'.
      - 'mean_cumulative_reward' column containing the row-wise mean of the 5 cumulative rewards.

    Parameters:
      df1, df2, df3, df4, df5 (pd.DataFrame): Data frames with columns 'episode' and 'cummulative_reward'.

    Returns:
      pd.DataFrame: The merged and aggregated data frame.
    """
    # Rename the 'cummulative_reward' column in each data frame to a unique name.
    df1_renamed = df1.rename(columns={'cumulative_reward': 'cumulative_reward_1'})
    df2_renamed = df2.rename(columns={'cumulative_reward': 'cumulative_reward_2'})
    df3_renamed = df3.rename(columns={'cumulative_reward': 'cumulative_reward_3'})
    df4_renamed = df4.rename(columns={'cumulative_reward': 'cumulative_reward_4'})
    df5_renamed = df5.rename(columns={'cumulative_reward': 'cumulative_reward_5'})

    # Merge the data frames on the 'episode' column.
    merged_df = df1_renamed[['episode', 'cumulative_reward_1']].copy()
    merged_df = merged_df.merge(df2_renamed[['episode', 'cumulative_reward_2']], on='episode')
    merged_df = merged_df.merge(df3_renamed[['episode', 'cumulative_reward_3']], on='episode')
    merged_df = merged_df.merge(df4_renamed[['episode', 'cumulative_reward_4']], on='episode')
    merged_df = merged_df.merge(df5_renamed[['episode', 'cumulative_reward_5']], on='episode')

    # Compute the episode-wise mean of the cumulative rewards.
    reward_columns = [
        'cumulative_reward_1',
        'cumulative_reward_2',
        'cumulative_reward_3',
        'cumulative_reward_4',
        'cumulative_reward_5'
    ]
    merged_df['mean_cumulative_reward'] = merged_df[reward_columns].mean(axis=1)

    return merged_df

CALCULATE FINAL MISALIGNMENT, AVERAGE TERMINATE TIME & AVERAGE EPISODIC REWARD

In [ ]:
def calculate_performance(value_1, value_2, value_3):
    mean_1 = sum(value_1) / len(value_1)
    mean_2 = sum(value_2) / len(value_2)
    mean_3 = sum(value_3) / len(value_3)

    return mean_1, mean_2, mean_3

0. MODEL TESTING (Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 2)

In [ ]:
trajectory_df_1 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/05_test_trajectories_edge_2/3_hf_d_conservative_edge_2/1_trajectory_hf_d_conservative_edge_2_df.pkl')    # Update directory location 1
trajectory_df_2 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/05_test_trajectories_edge_2/3_hf_d_conservative_edge_2/2_trajectory_hf_d_conservative_edge_2_df.pkl')    # Update directory location 2
trajectory_df_3 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/05_test_trajectories_edge_2/3_hf_d_conservative_edge_2/3_trajectory_hf_d_conservative_edge_2_df.pkl')    # Update directory location 3
trajectory_df_4 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/05_test_trajectories_edge_2/3_hf_d_conservative_edge_2/4_trajectory_hf_d_conservative_edge_2_df.pkl')    # Update directory location 4
trajectory_df_5 = pd.read_pickle('/content/drive/MyDrive/05_zero_shot_llm_3/02_data/05_test_trajectories_edge_2/3_hf_d_conservative_edge_2/5_trajectory_hf_d_conservative_edge_2_df.pkl')    # Update directory location 5

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


TESTING THE MODEL BY ANALYZING THE DATA FRAME <br>



1. Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 2 data frame 1<br>
trajectory_df_1 ---> 1_trajectory_hf_d_conservative_edge_2_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_1)

,episode,time_step,state,action,reward,next_state,collision_flag,lane_index
0,0,0,"[1.0, 0.88696575, 0.5, 0.3125, 0.0, 1.0, 0.098...",0,0.820735,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",0,1
1,0,1,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",0,0.798240,"[1.0, 1.0, 0.039846644, 0.31167492, -0.0226933...",0,0
2,0,2,"[1.0, 1.0, 0.039846644, 0.31167492, -0.0226933...",3,0.910539,"[1.0, 1.0, 0.0023418518, 0.36431533, -0.001740...",0,0
3,0,3,"[1.0, 1.0, 0.0023418518, 0.36431533, -0.001740...",2,0.950439,"[1.0, 1.0, 0.21463476, 0.37260154, 0.020676875...",0,1
4,0,4,"[1.0, 1.0, 0.21463476, 0.37260154, 0.020676875...",3,0.954884,"[1.0, 1.0, 0.24794306, 0.37468523, 0.001464026...",0,1
...,...,...,...,...,...,...,...,...
1595,99,11,"[1.0, 1.0, 0.7478184, 0.37499675, 0.0015564223...",0,0.976552,"[1.0, 1.0, 0.5357016, 0.37442532, -0.020752873...",0,2
1596,99,12,"[1.0, 1.0, 0.5357016, 0.37442532, -0.020752873...",0,0.954139,"[1.0, 1.0, 0.28786933, 0.37433589, -0.02230791...",0,1
1597,99,13,"[1.0, 1.0, 0.28786933, 0.37433589, -0.02230791...",3,0.955549,"[1.0, 1.0, 0.25217968, 0.37499678, -0.00155440...",0,1
1598,99,14,"[1.0, 1.0, 0.25217968, 0.37499678, -0.00155440...",3,0.955556,"[1.0, 1.0, 0.250086, 0.375, -7.0024566e-05, 1....",0,1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


A. MISALIGNMENT FOR DATA FRAME (trajectory_df_1)

In [ ]:
misalignment_df_1, misalignment_1 = calculate_misalignment(trajectory_df_1)
print("misalignment:", misalignment_1)

misalignment: 14.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_1)

,episode,Misalignment Score
0,0,14.0
1,1,14.0
2,2,14.0
3,3,14.0
4,4,14.0
...,...,...
95,95,14.0
96,96,14.0
97,97,14.0
98,98,14.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_1)

In [ ]:
collision_rate_df_1, collision_rate_1 = calculate_collision_rate(trajectory_df_1)
print("average terminate time:", collision_rate_1)

average terminate time: 16.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_1)

,Episode,Total Steps
0,0,16
1,1,16
2,2,16
3,3,16
4,4,16
...,...,...
95,95,16
96,96,16
97,97,16
98,98,16


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


C. STABILITY OF LEARNING CURVES (trajectory_df_1)

In [ ]:
learning_curve_result_df_1, variance_1, std_dev_1, final_reward_1 = analyze_learning_curve(trajectory_df_1)
print("Variance:", variance_1)
print("Standard Deviation:", std_dev_1)
print("average episodic reward:", final_reward_1)

Variance: 3.187316788771765e-28
Standard Deviation: 1.7853057969915867e-14
average episodic reward: 14.071186443727836


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_1)

,episode,cumulative_reward,smoothed_reward
0,0,14.071186,14.071186
1,1,14.071186,14.071186
2,2,14.071186,14.071186
3,3,14.071186,14.071186
4,4,14.071186,14.071186
...,...,...,...
95,95,14.071186,14.071186
96,96,14.071186,14.071186
97,97,14.071186,14.071186
98,98,14.071186,14.071186


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


2. Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 2 data frame 2<br>
trajectory_df_2 ---> 2_trajectory_hf_d_conservative_edge_2_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_2)

,episode,time_step,state,action,reward,next_state,collision_flag,lane_index
0,0,0,"[1.0, 0.91813225, 0.5, 0.3125, 0.0, 1.0, 0.100...",3,0.954993,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,2
1,0,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,0.950430,"[1.0, 1.0, 0.28544697, 0.3725976, -0.02074788,...",0,1
2,0,2,"[1.0, 1.0, 0.28544697, 0.3725976, -0.02074788,...",3,0.288217,"[1.0, 1.0, 0.2513637, 0.37468523, -0.001465738...",1,1
3,1,0,"[1.0, 0.91813225, 0.5, 0.3125, 0.0, 1.0, 0.100...",3,0.954993,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,2
4,1,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,0.950430,"[1.0, 1.0, 0.28544697, 0.3725976, -0.02074788,...",0,1
...,...,...,...,...,...,...,...,...
295,98,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,0.950430,"[1.0, 1.0, 0.28544697, 0.3725976, -0.02074788,...",0,1
296,98,2,"[1.0, 1.0, 0.28544697, 0.3725976, -0.02074788,...",3,0.288217,"[1.0, 1.0, 0.2513637, 0.37468523, -0.001465738...",1,1
297,99,0,"[1.0, 0.91813225, 0.5, 0.3125, 0.0, 1.0, 0.100...",3,0.954993,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,2
298,99,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.091804...",0,0.950430,"[1.0, 1.0, 0.28544697, 0.3725976, -0.02074788,...",0,1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


A. MISALIGNMENT FOR DATA FRAME (trajectory_df_2)

In [ ]:
misalignment_df_2, misalignment_2 = calculate_misalignment(trajectory_df_2)
print("misalignment:", misalignment_2)

misalignment: 6.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_2)

,episode,Misalignment Score
0,0,6.0
1,1,6.0
2,2,6.0
3,3,6.0
4,4,6.0
...,...,...
95,95,6.0
96,96,6.0
97,97,6.0
98,98,6.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_2)

In [ ]:
collision_rate_df_2, collision_rate_2 = calculate_collision_rate(trajectory_df_2)
print("average terminate time:", collision_rate_2)

average terminate time: 3.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_2)

,Episode,Total Steps
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3
...,...,...
95,95,3
96,96,3
97,97,3
98,98,3


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


C. STABILITY OF LEARNING CURVES (trajectory_df_2)

In [ ]:
learning_curve_result_df_2, variance_2, std_dev_2, final_reward_2 = analyze_learning_curve(trajectory_df_2)
print("Variance:", variance_2)
print("Standard Deviation:", std_dev_2)
print("average episodic reward:", final_reward_2)

Variance: 2.8685851098945886e-29
Standard Deviation: 5.355917390974761e-15
average episodic reward: 2.193640512691174


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_2)

,episode,cumulative_reward,smoothed_reward
0,0,2.193641,2.193641
1,1,2.193641,2.193641
2,2,2.193641,2.193641
3,3,2.193641,2.193641
4,4,2.193641,2.193641
...,...,...,...
95,95,2.193641,2.193641
96,96,2.193641,2.193641
97,97,2.193641,2.193641
98,98,2.193641,2.193641


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


3. Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 2 data frame 3<br>
trajectory_df_3 ---> 3_trajectory_hf_d_conservative_edge_2_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_3)

,episode,time_step,state,action,reward,next_state,collision_flag,lane_index
0,0,0,"[1.0, 0.8993783, 0.5, 0.3125, 0.0, 1.0, 0.1008...",3,0.954993,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",0,2
1,0,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",3,0.973884,"[1.0, 1.0, 0.5, 0.37317482, 0.0, 1.0, 0.006382...",0,2
2,0,2,"[1.0, 1.0, 0.5, 0.37317482, 0.0, 1.0, 0.006382...",2,0.066667,"[1.0, 1.0, 0.71271163, 0.21317253, 0.03141851,...",1,3
3,1,0,"[1.0, 0.8993783, 0.5, 0.3125, 0.0, 1.0, 0.1008...",3,0.954993,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",0,2
4,1,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",3,0.973884,"[1.0, 1.0, 0.5, 0.37317482, 0.0, 1.0, 0.006382...",0,2
...,...,...,...,...,...,...,...,...
295,98,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",3,0.973884,"[1.0, 1.0, 0.5, 0.37317482, 0.0, 1.0, 0.006382...",0,2
296,98,2,"[1.0, 1.0, 0.5, 0.37317482, 0.0, 1.0, 0.006382...",2,0.066667,"[1.0, 1.0, 0.71271163, 0.21317253, 0.03141851,...",1,3
297,99,0,"[1.0, 0.8993783, 0.5, 0.3125, 0.0, 1.0, 0.1008...",3,0.954993,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",0,2
298,99,1,"[1.0, 1.0, 0.5, 0.36431947, 0.0, 1.0, 0.064678...",3,0.973884,"[1.0, 1.0, 0.5, 0.37317482, 0.0, 1.0, 0.006382...",0,2


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


A. MISALIGNMENT FOR DATA FRAME (trajectory_df_3)

In [ ]:
misalignment_df_3, misalignment_3 = calculate_misalignment(trajectory_df_3)
print("misalignment:", misalignment_3)

misalignment: 6.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_3)

,episode,Misalignment Score
0,0,6.0
1,1,6.0
2,2,6.0
3,3,6.0
4,4,6.0
...,...,...
95,95,6.0
96,96,6.0
97,97,6.0
98,98,6.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_3)

In [ ]:
collision_rate_df_3, collision_rate_3 = calculate_collision_rate(trajectory_df_3)
print("average terminate time:", collision_rate_3)

average terminate time: 3.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_3)

,Episode,Total Steps
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3
...,...,...
95,95,3
96,96,3
97,97,3
98,98,3


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


C. STABILITY OF LEARNING CURVES (trajectory_df_3)

In [ ]:
learning_curve_result_df_3, variance_3, std_dev_3, final_reward_3 = analyze_learning_curve(trajectory_df_3)
print("Variance:", variance_3)
print("Standard Deviation:", std_dev_3)
print("average episodic reward:", final_reward_3)

Variance: 1.2450456206139707e-30
Standard Deviation: 1.1158161231197417e-15
average episodic reward: 1.9955434191075845


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_3)

,episode,cumulative_reward,smoothed_reward
0,0,1.995543,1.995543
1,1,1.995543,1.995543
2,2,1.995543,1.995543
3,3,1.995543,1.995543
4,4,1.995543,1.995543
...,...,...,...
95,95,1.995543,1.995543
96,96,1.995543,1.995543
97,97,1.995543,1.995543
98,98,1.995543,1.995543


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


4. Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 4 data frame 2<br>
trajectory_df_4 ---> 4_trajectory_hf_d_conservative_edge_2_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_4)

,episode,time_step,state,action,reward,next_state,collision_flag,lane_index
0,0,0,"[1.0, 0.8877945, 0.5, 0.3125, 0.0, 1.0, 0.1016...",0,0.820735,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",0,1
1,0,1,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",2,0.843206,"[1.0, 1.0, 0.4658618, 0.31191945, 0.019039502,...",0,2
2,0,2,"[1.0, 1.0, 0.4658618, 0.31191945, 0.019039502,...",2,0.864926,"[1.0, 1.0, 0.7103472, 0.31168425, 0.022564989,...",0,3
3,0,3,"[1.0, 1.0, 0.7103472, 0.31168425, 0.022564989,...",3,0.066667,"[1.0, 1.0, 0.75200003, 0.20010604, 0.004482114...",1,3
4,1,0,"[1.0, 0.8877945, 0.5, 0.3125, 0.0, 1.0, 0.1016...",0,0.820735,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",0,1
...,...,...,...,...,...,...,...,...
395,98,3,"[1.0, 1.0, 0.7103472, 0.31168425, 0.022564989,...",3,0.066667,"[1.0, 1.0, 0.75200003, 0.20010604, 0.004482114...",1,3
396,99,0,"[1.0, 0.8877945, 0.5, 0.3125, 0.0, 1.0, 0.1016...",0,0.820735,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",0,1
397,99,1,"[1.0, 1.0, 0.28698906, 0.31180274, -0.02086363...",2,0.843206,"[1.0, 1.0, 0.4658618, 0.31191945, 0.019039502,...",0,2
398,99,2,"[1.0, 1.0, 0.4658618, 0.31191945, 0.019039502,...",2,0.864926,"[1.0, 1.0, 0.7103472, 0.31168425, 0.022564989,...",0,3


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


A. MISALIGNMENT FOR DATA FRAME (trajectory_df_4)

In [ ]:
misalignment_df_4, misalignment_4 = calculate_misalignment(trajectory_df_4)
print("misalignment:", misalignment_4)

misalignment: 7.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_4)

,episode,Misalignment Score
0,0,7.0
1,1,7.0
2,2,7.0
3,3,7.0
4,4,7.0
...,...,...
95,95,7.0
96,96,7.0
97,97,7.0
98,98,7.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_4)

In [ ]:
collision_rate_df_4, collision_rate_4 = calculate_collision_rate(trajectory_df_4)
print("average terminate time:", collision_rate_4)

average terminate time: 4.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_4)

,Episode,Total Steps
0,0,4
1,1,4
2,2,4
3,3,4
4,4,4
...,...,...
95,95,4
96,96,4
97,97,4
98,98,4


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


C. STABILITY OF LEARNING CURVES (trajectory_df_4)

In [ ]:
learning_curve_result_df_4, variance_4, std_dev_4, final_reward_4 = analyze_learning_curve(trajectory_df_4)
print("Variance:", variance_4)
print("Standard Deviation:", std_dev_4)
print("average episodic reward:", final_reward_4)

Variance: 1.613579124315706e-29
Standard Deviation: 4.01693804323107e-15
average episodic reward: 2.5955337854012908


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_4)

,episode,cumulative_reward,smoothed_reward
0,0,2.595534,2.595534
1,1,2.595534,2.595534
2,2,2.595534,2.595534
3,3,2.595534,2.595534
4,4,2.595534,2.595534
...,...,...,...
95,95,2.595534,2.595534
96,96,2.595534,2.595534
97,97,2.595534,2.595534
98,98,2.595534,2.595534


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


5. Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 2 data frame 5<br>
trajectory_df_5 ---> 5_trajectory_hf_d_conservative_edge_2_df

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df_5)

,episode,time_step,state,action,reward,next_state,collision_flag,lane_index
0,0,0,"[1.0, 0.87540454, 0.5, 0.3125, 0.0, 1.0, 0.105...",0,0.820735,"[1.0, 0.9991162, 0.28698906, 0.31180274, -0.02...",0,1
1,0,1,"[1.0, 0.9991162, 0.28698906, 0.31180274, -0.02...",0,0.798240,"[1.0, 1.0, 0.039846644, 0.31167492, -0.0226933...",0,0
2,0,2,"[1.0, 1.0, 0.039846644, 0.31167492, -0.0226933...",3,0.910539,"[1.0, 1.0, 0.0023418518, 0.36431533, -0.001740...",0,0
3,0,3,"[1.0, 1.0, 0.0023418518, 0.36431533, -0.001740...",2,0.022222,"[1.0, 1.0, 0.25109962, 0.18273093, 0.029716175...",1,1
4,1,0,"[1.0, 0.87540454, 0.5, 0.3125, 0.0, 1.0, 0.105...",0,0.820735,"[1.0, 0.9991162, 0.28698906, 0.31180274, -0.02...",0,1
...,...,...,...,...,...,...,...,...
395,98,3,"[1.0, 1.0, 0.0023418518, 0.36431533, -0.001740...",2,0.022222,"[1.0, 1.0, 0.25109962, 0.18273093, 0.029716175...",1,1
396,99,0,"[1.0, 0.87540454, 0.5, 0.3125, 0.0, 1.0, 0.105...",0,0.820735,"[1.0, 0.9991162, 0.28698906, 0.31180274, -0.02...",0,1
397,99,1,"[1.0, 0.9991162, 0.28698906, 0.31180274, -0.02...",0,0.798240,"[1.0, 1.0, 0.039846644, 0.31167492, -0.0226933...",0,0
398,99,2,"[1.0, 1.0, 0.039846644, 0.31167492, -0.0226933...",3,0.910539,"[1.0, 1.0, 0.0023418518, 0.36431533, -0.001740...",0,0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


A. MISALIGNMENT FOR DATA FRAME (trajectory_df_5)

In [ ]:
misalignment_df_5, misalignment_5 = calculate_misalignment(trajectory_df_5)
print("misalignment:", misalignment_5)

misalignment: 7.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(misalignment_df_5)

,episode,Misalignment Score
0,0,7.0
1,1,7.0
2,2,7.0
3,3,7.0
4,4,7.0
...,...,...
95,95,7.0
96,96,7.0
97,97,7.0
98,98,7.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


B. AVERAGE TERMINATE TIME FOR DATA FRAME (trajectory_df_5)

In [ ]:
collision_rate_df_5, collision_rate_5 = calculate_collision_rate(trajectory_df_5)
print("average terminate time:", collision_rate_5)

average terminate time: 4.0


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(collision_rate_df_5)

,Episode,Total Steps
0,0,4
1,1,4
2,2,4
3,3,4
4,4,4
...,...,...
95,95,4
96,96,4
97,97,4
98,98,4


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


C. STABILITY OF LEARNING CURVES (trajectory_df_5)

In [ ]:
learning_curve_result_df_5, variance_5, std_dev_5, final_reward_5 = analyze_learning_curve(trajectory_df_5)
print("Variance:", variance_5)
print("Standard Deviation:", std_dev_5)
print("average episodic reward:", final_reward_5)

Variance: 9.76115766561353e-30
Standard Deviation: 3.124285144735277e-15
average episodic reward: 2.551736205912338


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(learning_curve_result_df_5)

,episode,cumulative_reward,smoothed_reward
0,0,2.551736,2.551736
1,1,2.551736,2.551736
2,2,2.551736,2.551736
3,3,2.551736,2.551736
4,4,2.551736,2.551736
...,...,...,...
95,95,2.551736,2.551736
96,96,2.551736,2.551736
97,97,2.551736,2.551736
98,98,2.551736,2.551736


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


COMBINED DATA FRAMES - Biased_HF_D_Conservative Edge Case 2

In [ ]:
cummulative_reward = create_cummulative_reward(
        learning_curve_result_df_1,
        learning_curve_result_df_2,
        learning_curve_result_df_3,
        learning_curve_result_df_4,
        learning_curve_result_df_5
    )

data_table.enable_dataframe_formatter()
data_table.DataTable(cummulative_reward)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,episode,cumulative_reward_1,cumulative_reward_2,cumulative_reward_3,cumulative_reward_4,cumulative_reward_5,mean_cumulative_reward
0,0,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
1,1,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
2,2,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
3,3,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
4,4,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
...,...,...,...,...,...,...,...
95,95,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
96,96,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
97,97,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528
98,98,14.071186,2.193641,1.995543,2.595534,2.551736,4.681528


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


FINAL MISALIGNMENT, AVERAGE TERMINATE TIME & AVERAGE EPISODIC REWARD : Biased_HF_D_RECKLESS_ADAPTIVE Edge Case 2

In [ ]:
 # Organize the values into lists
misalignment_list_1 = [misalignment_1, misalignment_2, misalignment_3, misalignment_4, misalignment_5]
average_terminate_time_list_1 = [collision_rate_1, collision_rate_2, collision_rate_3, collision_rate_4, collision_rate_5]
average_episodic_reward_list_1 = [final_reward_1, final_reward_2, final_reward_3, final_reward_4, final_reward_5]

# Call the function to compute the mean performance values
mean_misalignment_1, mean_average_terminate_time_1 , mean_average_episodic_reward_1 = calculate_performance(misalignment_list_1, average_terminate_time_list_1, average_episodic_reward_list_1)

# Print the results
print("BIASED_HF_D_RAD EDGE CASE 2 FINAL MISALIGNMENT:", mean_misalignment_1)
print("BIASED_HF_D_RAD EDGE CASE 2 TERMINATE TIME:", mean_average_terminate_time_1)
print("BIASED_HF_D_RAD EDGE CASE 2 AVERAGE EPISODIC REWARD:", mean_average_episodic_reward_1)

BIASED_HF_D_RAD EDGE CASE 2 FINAL MISALIGNMENT: 8.0
BIASED_HF_D_RAD EDGE CASE 2 TERMINATE TIME: 6.0
BIASED_HF_D_RAD EDGE CASE 2 AVERAGE EPISODIC REWARD: 4.681528073368044
